# Python - Oracle 연동

### 패키지 참조

- 사용 패키지 : cx_Oracle, sqlalchemy, pandas (+ openpyxl, xlrd)

In [1]:
import cx_Oracle as cx
from sqlalchemy import create_engine
from pandas import DataFrame, read_sql_table

### cx_Oracle을 사용한 연동

1. 데이터베이스 접속 및 cursor 객체 생성

In [2]:
dbcon = cx.connect("hr", "hr", "localhost:1521/xe")
cursor = dbcon.cursor()

2. 데이터 저장하기

- 저장을 위한 값 입력받기

In [3]:
dname = input("학과 이름을 입력하세요.")
loc = input("학과의 위치를 입력하세요.")
dname, loc

('테스트학과', '여기')

- SQL문은 홑따옴표로 문자열 구분을 판단하는데 이 헛점을 이용하여 SQL문을 의도와 다르게 입력값을 주어 해킹을 하는 기법을 SQL 인젝션 공격이라고 함
  - 오라클은 해당 취약점을 수정했으나 MySQL은 해당 헛점 존재
  - 이 해킹 방법을 방지하기 위해서는 입력 홑따옴표를 실제 홑따옴표로 인식하도록 replace로 '를 \\\\'로 바꾸면 됨

- 저장을 위한 새로운 시퀀스값 조회

In [4]:
sql = "SELECT seq_department.nextval FROM dual"
cursor.execute(sql)
result = cursor.fetchall()
print(result)

seq = result[0][0]
print("새로운 시퀀스 번호 : %d" % seq)

[(340,)]
새로운 시퀀스 번호 : 340


3. INSERT문 수행

In [5]:
sql = "INSERT INTO department (deptno, dname, loc) VALUES (:1, :2, :3)"

#                     :1    :2    :3
cursor.execute(sql, [seq, dname, loc])
print("%s개의 행이 저장됨" % cursor.rowcount)

# 처리 결과를 실제로 반영함
dbcon.commit()

# 되돌리기
# -> 이미 commit()한 내역은 적용되지 않음
# dbcon.rollback()

1개의 행이 저장됨


4. UPDATE문 수행

In [7]:
sql = "UPDATE department SET dname=:1, loc=:2 WHERE deptno=:3"
cursor.execute(sql, ['메가스터디', '신촌', seq])
print("%s개의 행이 갱신됨" % cursor.rowcount)
dbcon.commit()

1개의 행이 갱신됨


5. DELETE문 수행

In [8]:
sql = "DELETE FROM department WHERE deptno=:1"
cursor.execute(sql, [seq])
print("%s개의 행이 삭제됨" % cursor.rowcount)
dbcon.commit()

1개의 행이 삭제됨


6. 데이터베이스 접속 해제

In [9]:
try:
    cursor.close()
    dbcon.close()
except Exception as e:
    print("접속 상태가 아닙니다", e)

- 연구과제

In [ ]:
dbcon = cx.connect("hr", "hr", "localhost:1521/xe")
cursor = dbcon.cursor()

In [25]:
with open("traffic_acc.csv", "r", encoding = "utf-8") as f:
    csv = f.readlines()

data = []

for i, v in enumerate(csv):
    if i == 0:
        continue

    line = v.strip().split(",")
    item = {"년도":int(line[0]), "월":int(line[1]), "발생건수":int(line[2]), "사망자수":int(line[3]), "부상자수":int(line[4]),}
    data.append(item)

In [42]:
sql = "CREATE TABLE Traffic_acc(id number not null, 년도 number not null, 월 number not null, 발생건수 number not null, 사망자수 number not null, 부상자수 number not null, primary key(id))"
cursor.execute(sql)

In [46]:
sql = "CREATE SEQUENCE seq_acc start with 100 increment by 1"
cursor.execute(sql)

In [48]:
for i in data:
    sql = "INSERT INTO Traffic_acc (id, 년도, 월, 발생건수, 사망자수, 부상자수) VALUES (seq_acc.nextval, :1, :2, :3, :4, :5)"
    cursor.execute(sql, [i['년도'], i['월'], i['발생건수'], i['사망자수'], i['부상자수']])
dbcon.commit()

In [49]:
cursor.close()
dbcon.close()